In [1]:
import pandas as pd

In [2]:
LINK = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"

### pipe

In [3]:
def get_table(link:str) -> pd.DataFrame:
    html_data:list[pd.DataFrame] = pd.read_html(LINK)
    return html_data[0]

In [21]:
def dropna(df:pd.DataFrame, drop_based_on:str) -> pd.DataFrame:
    return df.dropna(subset=drop_based_on).copy()

In [14]:
def drop_columns(df:pd.DataFrame, columns_to_drop:list[str]) -> pd.DataFrame:
    return df.drop(columns=columns_to_drop)

In [15]:
def extract_teams_score(df:pd.DataFrame, score_column:str) -> pd.DataFrame:
    df[['home_score', 'away_score']] = df[score_column].str.split(pat='–',expand=True).astype('float16')
    return df

In [16]:
def calculate_teams_points(home_score:float, away_score:float, calc_for_team:str='home') -> pd.DataFrame:
    if calc_for_team == 'home':
        if home_score > away_score:
            return 3
        if home_score == away_score:
            return 1
        return 0 
    if calc_for_team == 'away':
        if home_score < away_score:
            return 3
        if home_score == away_score:
            return 1
        return 0 

In [17]:
def extract_teams_points(df:pd.DataFrame, home_score_column:str, away_score_columns:str, calc_for_team:str='home') -> pd.DataFrame:
    temp_series =  df[[home_score_column, away_score_columns]].apply(lambda row: calculate_teams_points(row[home_score_column], row[away_score_columns], calc_for_team), axis=1)
    df[f'{calc_for_team}_points'] = pd.DataFrame(temp_series)
    return df

In [8]:
def create_datatime(df:pd.DataFrame, date_column:str, time_column:str):
    df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    return df

In [22]:
pl_table = get_table(LINK)
pl_table = (
pl_table
    .pipe(dropna, 'Score')
    .pipe(extract_teams_score, 'Score')
    .pipe(extract_teams_points, 'home_score', 'away_score', 'home')
    .pipe(extract_teams_points, 'home_score', 'away_score', 'away')
    .pipe(create_datatime, 'Date', 'Time')
    .pipe(drop_columns, ['Wk', 'Day', 'Venue', 'Referee', 'Match Report', 'Notes', 'Date', 'Time'])

)

In [23]:
pl_table


,Home,xG,Score,xG.1,Away,Attendance,home_score,away_score,home_points,away_points,datetime
0,Burnley,0.3,0–3,1.9,Manchester City,21572.0,0.0,3.0,0,3,2023-08-11 20:00:00
1,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,2.0,1.0,3,0,2023-08-12 12:30:00
2,Everton,2.7,0–1,1.5,Fulham,39940.0,0.0,1.0,0,3,2023-08-12 15:00:00
3,Sheffield Utd,0.5,0–1,1.9,Crystal Palace,31194.0,0.0,1.0,0,3,2023-08-12 15:00:00
4,Brighton,4.0,4–1,1.5,Luton Town,31872.0,4.0,1.0,3,0,2023-08-12 15:00:00
5,Bournemouth,1.3,1–1,1.1,West Ham,11245.0,1.0,1.0,1,1,2023-08-12 15:00:00
6,Newcastle Utd,3.3,5–1,1.8,Aston Villa,52207.0,5.0,1.0,3,0,2023-08-12 17:30:00
7,Brentford,2.2,2–2,1.3,Tottenham,17066.0,2.0,2.0,1,1,2023-08-13 14:00:00
8,Chelsea,1.4,1–1,1.3,Liverpool,40096.0,1.0,1.0,1,1,2023-08-13 16:30:00
9,Manchester Utd,2.2,1–0,2.2,Wolves,73358.0,1.0,0.0,3,0,2023-08-14 20:00:00


In [66]:
def create_race_table(df:pd.DataFrame) -> pd.DataFrame:
    teams_set:set[str] = {*pl_table['Home'].unique().tolist(), *pl_table['Away'].unique().tolist(),}
    for team in teams_set:

        new_tables_list:list[pd.DataFrame] = []
        team_table:pd.DataFrame =  pl_table.query(f' Home == "{team}" or Away == "{team}" ').copy()
        match_order:pd.Series = team_table['datetime'].rank()
        points:pd.Series = team_table.apply(lambda row: row['home_points'] if row['Home'] == '{team}' else row['away_points'], axis=1)
        points_cum:pd.Series = points.cumsum()

        new_tables_list.append(
            pd.DataFrame({
            'team_nam': team,
            'match_order':match_order,
            'points': points,
            'points_cum': points_cum,
            }))

               
        return  
        print(team)

create_race_table(pl_table)

,team_nam,match_order,points,points_cum
1,Nott'ham Forest,1.0,0,0
11,Nott'ham Forest,2.0,0,0
25,Nott'ham Forest,3.0,0,0
34,Nott'ham Forest,4.0,3,3
52,Nott'ham Forest,5.0,1,4
54,Nott'ham Forest,6.0,0,4


In [36]:
arsenal_table = pl_table.query(f' Home == "Arsenal" or Away == "Arsenal" ').copy()

In [61]:
team_name = 'Arsenal'
match_order:pd.Series = arsenal_table['datetime'].rank()
points:pd.Series = arsenal_table.apply(lambda row: row['home_points'] if row['Home'] == 'Arsenal' else row['away_points'], axis=1)
points_cum:pd.Series = points.cumsum()

pd.DataFrame({
    'team_nam':'Arsenal',
    'match_order':match_order,
    'points': points,
    'points_cum': points_cum,
})

,team_nam,match_order,points,points_cum
1,Arsenal,1.0,3,3
19,Arsenal,2.0,3,6
26,Arsenal,3.0,1,7
41,Arsenal,4.0,3,10
51,Arsenal,5.0,3,13


,team_nam,match_order,points,points_cum
1,Arsenal,1.0,3,3
19,Arsenal,2.0,3,6
26,Arsenal,3.0,1,7
41,Arsenal,4.0,3,10
51,Arsenal,5.0,3,13


In [53]:
pd.concat([match_order, points]).transpose()

1     1.0
19    2.0
26    3.0
41    4.0
51    5.0
1     3.0
19    3.0
26    1.0
41    3.0
51    3.0
dtype: float64

In [ ]:
team_name = 'Arsenal'
arsenal_table['match_order'] = arsenal_table['datetime'].rank()

arsenal_table['points'] = arsenal_table.apply(lambda row: row['home_points'] if row['Home'] == 'Arsenal' else row['away_points'], axis=1)
arsenal_table['points_cum'] = arsenal_table['points'].cumsum()
arsenal_table

In [12]:
arsenal_table['team'] = 'Arsenal'
arsenal_table['match_order'] = arsenal_table['datetime'].rank()
arsenal_table['points'] = arsenal_table['home_points'] + arsenal_table['away_points']

arsenal_table['points'] = arsenal_table.apply(lambda row: row['home_points'] if row['Home'] == 'Arsenal' else row['away_points'], axis=1)
arsenal_table['points_cum'] = arsenal_table['points'].cumsum()
arsenal_table

,Home,xG,Score,xG.1,Away,Attendance,home_score,away_score,home_points,away_points,datetime,team,match_order,points,points_cum
1,Arsenal,0.8,2–1,1.2,Nott'ham Forest,59984.0,2.0,1.0,3,0,2023-08-12 12:30:00,Arsenal,1.0,3,3
19,Crystal Palace,1.0,0–1,2.0,Arsenal,24189.0,0.0,1.0,0,3,2023-08-21 20:00:00,Arsenal,2.0,3,6
26,Arsenal,3.2,2–2,0.6,Fulham,59961.0,2.0,2.0,1,1,2023-08-26 15:00:00,Arsenal,3.0,1,7
41,Arsenal,2.3,3–1,0.9,Manchester Utd,60192.0,3.0,1.0,3,0,2023-09-03 16:30:00,Arsenal,4.0,3,10
51,Everton,0.3,0–1,1.0,Arsenal,39217.0,0.0,1.0,0,3,2023-09-17 16:30:00,Arsenal,5.0,3,13
59,Arsenal,NaN,NaN,NaN,Tottenham,NaN,NaN,NaN,0,0,2023-09-24 14:00:00,Arsenal,6.0,0,13
67,Bournemouth,NaN,NaN,NaN,Arsenal,NaN,NaN,NaN,0,0,2023-09-30 15:00:00,Arsenal,7.0,0,13
87,Arsenal,NaN,NaN,NaN,Manchester City,NaN,NaN,NaN,0,0,2023-10-08 16:30:00,Arsenal,8.0,0,13
95,Chelsea,NaN,NaN,NaN,Arsenal,NaN,NaN,NaN,0,0,2023-10-21 17:30:00,Arsenal,9.0,0,13
103,Arsenal,NaN,NaN,NaN,Sheffield Utd,NaN,NaN,NaN,0,0,2023-10-28 15:00:00,Arsenal,10.0,0,13
